# Menganalisis Risiko Gagal Bayar Peminjam

Kali ini saya akan mencari tahu pengaruh status perkawinan seorang nasabah dan jumlah anak yang dimilikinya terhadap probabilitas gagal bayar dalam pelunasan pinjaman. Pihak bank sudah memiliki beberapa data mengenai kelayakan kredit nasabah.

Laporan ini akan dipertimbangkan pada saat membuat **penilaian kredit** untuk calon nasabah. **Penilaian kredit** digunakan untuk mengevaluasi kemampuan calon peminjam untuk melunasi pinjaman mereka.

In [88]:
import pandas as pd
import numpy as np

In [89]:
try:
    df = pd.read_csv('credit_scoring_eng.csv')# Muat datanya
except:
    df = pd.read_csv('/datasets/credit_scoring_eng.csv')

## Soal 1. Eksplorasi data

**Deskripsi Data**
- `children` - jumlah anak dalam keluarga
- `days_employed` - pengalaman kerja nasabah dalam hari
- `dob_years` - usia nasabah dalam tahun
- `education` - tingkat pendidikan nasabah
- `education_id` - pengidentifikasi untuk tingkat pendidikan nasabah
- `family_status` - pengidentifikasi untuk status perkawinan nasabah
- `family_status_id` - tanda pengenal status perkawinan
- `gender` - jenis kelamin nasabah
- `income_type` - jenis pekerjaan
- `debt` - apakah nasabah memiliki hutang pembayaran pinjaman
- `total_income` - pendapatan bulanan
- `purpose` - tujuan mendapatkan pinjaman

In [90]:
df.shape# Mari kita lihat berapa banyak baris dan kolom yang dimiliki oleh dataset kita



(21525, 12)

In [91]:
df.head()# Mari tampilkan N baris pertama



,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


Terdapat masalah pada kolom days_employed yang memuat nilai negatif pada hitungan hari kerja, serta kesalahan penulisan pada nilai di kolom education.

In [92]:
df.info()# Dapatkan informasi data


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Ada 2 kolom yang memiliki nilai yang hilang, yaitu kolom days_employed dan total_income. Perlu diamati lebih lanjut lagi.

In [93]:
# Mari kita lihat tabel yang difilter dengan nilai yang hilang di kolom pertama yang mengandung data yang hilang
df.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Jumlah nilai yang hilang adalah 2174. Karena total nilai adalah 21525. Perlu penyelidikan lebih lanjut untuk mengetahui apakah nilai yang hilang memang dibutuhkan atau tidak.

In [94]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,19351.000000
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,26787.568355
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,16475.450632
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,3306.762000
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,16488.504500
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,23202.870000
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,32549.611000
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,362496.645000


In [95]:
# Mari kita terapkan beberapa kondisi untuk memfilter data dan melihat jumlah baris dalam tabel yang telah difilter.
df[df['days_employed'].isnull() & df['total_income'].isnull()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


**Kesimpulan sementara**

Jumlah baris yg telah difilter sudah sesuai dengan jumlah nilai yang hilang. Selanjutnya kita dapat mengisi atau menghapus nilai yang hilang tersebut.

Dengan adanya hitungan persentase pada sel dibawah ini, nilai yang hilang berjumlah sekitar 10%.

Saya akan memilah kembali data/nilai mana yang perlu dihapus atau diisi. Karena bisa jadi nilai yang akan dihapus justru memiliki nilai penting berkaitan dengan data lainnya.


In [96]:
df['days_employed'].isnull().value_counts(normalize=True)

False    0.899001
True     0.100999
Name: days_employed, dtype: float64

In [97]:
df['total_income'].isnull().value_counts(normalize=True)

False    0.899001
True     0.100999
Name: total_income, dtype: float64

In [99]:
# Mari kita periksa nasabah yang tidak memiliki data tentang karakteristik yang teridentifikasi dan kolom dengan nilai yang hilang
df_null = df[df['days_employed'].isnull() & df['total_income'].isnull()].reset_index(drop=True)


In [100]:
df_null

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
1,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
2,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
3,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
4,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
2169,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
2170,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
2171,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
2172,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


In [101]:
df_null['education'].value_counts()# Periksalah distribusinya



secondary education    1408
bachelor's degree       496
SECONDARY EDUCATION      67
Secondary Education      65
some college             55
Bachelor's Degree        25
BACHELOR'S DEGREE        23
primary education        19
Some College              7
SOME COLLEGE              7
PRIMARY EDUCATION         1
Primary Education         1
Name: education, dtype: int64

Dikarenakan perbedaan gaya penulisan, informasi yang didapatkan dari kolom 'education' terlihat berantakan.

**Kemungkinan penyebab hilangnya nilai dalam data**

Saat ini masih belum bisa disimpulkan dengan pasti alasan hilangnya nilai tersebut, kita akan mengetahuinya dengan memperbaikinya terlebih dulu.

In [102]:
df['education'].value_counts()# Memeriksa distribusi di seluruh *dataset*



secondary education    13750
bachelor's degree       4718
SECONDARY EDUCATION      772
Secondary Education      711
some college             668
BACHELOR'S DEGREE        274
Bachelor's Degree        268
primary education        250
Some College              47
SOME COLLEGE              29
PRIMARY EDUCATION         17
Primary Education         15
graduate degree            4
Graduate Degree            1
GRADUATE DEGREE            1
Name: education, dtype: int64

**Kesimpulan sementara**

Apakah distribusi dalam *dataset* yang asli mirip dengan distribusi tabel yang telah difilter?
Tidak mirip, ini berarti data yang kita miliki harus dibersihkan dan dirapikan.

Mari pikirkan alasan lain yang dapat menyebabkan data hilang dan periksa apakah kita dapat menemukan pola tertentu yang dapat membuat kita berpikir bahwa hilangnya nilai-nilai tersebut tidak terjadi secara acak.

In [103]:
# Periksa penyebab dan pola lain yang dapat mengakibatkan nilai yang hilang
df_null

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
1,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
2,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
3,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
4,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
2169,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
2170,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
2171,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
2172,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


In [104]:
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions
21521,0,343937.404131,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car
21522,1,-2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property
21523,3,-3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car


Umumnya nilai yg hilang bisa berada di baris berbeda,namun yang terjadi di datasets ini bisa dilihat
dari hasil tampilan df dan df_null diatas, bahwa nilai yang hilang pada baris tertentu di kolom days_employed,
juga hilang pada baris yang sama di kolom total_income. 

**Kesimpulan sementara**

Nilai yang hilang pada kedua kolom memiliki jumlah yang sama dan di baris yang sama juga. Ini bisa jadi kebetulan, bisa juga karena disebabkan kesalahan atau eror saat menginput dan memindahkan data.

## Transformasi data

Mari kita perhatikan setiap kolom untuk melihat masalah apa yang mungkin dimiliki mereka.

Saya akan mulailah dengan menghapus duplikat dan memperbaiki data tentang informasi pendidikan jika diperlukan.

In [15]:
# Mari kita lihat semua nilai di kolom pendidikan untuk memeriksa ejaan apa yang perlu diperbaiki
df['education'].value_counts()

secondary education    13750
bachelor's degree       4718
SECONDARY EDUCATION      772
Secondary Education      711
some college             668
BACHELOR'S DEGREE        274
Bachelor's Degree        268
primary education        250
Some College              47
SOME COLLEGE              29
PRIMARY EDUCATION         17
Primary Education         15
graduate degree            4
GRADUATE DEGREE            1
Graduate Degree            1
Name: education, dtype: int64

In [16]:
df['education'] = df['education'].str.lower()# Perbaiki pencatatan jika diperlukan

In [17]:
df['education'].value_counts()
# Periksa semua nilai di kolom untuk memastikan bahwa kita telah memperbaikinya dengan tepat



secondary education    15233
bachelor's degree       5260
some college             744
primary education        282
graduate degree            6
Name: education, dtype: int64

Saya melihat banyaknya kesalahan dalam penulisan di kolom 'education', seperti perbedaan pada huruf kecil dan besar padahal nama status pendidikannya sama.
Maka dari itu, saya mengubahnya menjadi huruf kecil semua dengan menggunakan str.lower()

In [18]:
df['children'].value_counts()# Mari kita lihat distribusi nilai pada kolom `children'

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Tidak mungkin ada jumlah anak sebanyak -1, dan kecil juga kemungkinan mempunyai jumlah anak sebanyak 20. Saya akan menganggap masalah ini sebagai kesalahan dalam penginputan data, maka jumlah anak -1 akan saya ganti dengan 0, dan jumlah anak 20 akan saya ganti dengan 2.

In [19]:
# memperbaiki data berdasarkan keputusan
df['children'] = df['children'].replace({-1 : 0, 20 : 2})

In [20]:
# Periksa kembali kolom `children` untuk memastikan bahwa semuanya telah diperbaiki
df['children'].value_counts()


0    14196
1     4818
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

In [21]:
df['days_employed'].value_counts()

-327.685916     1
-1580.622577    1
-4122.460569    1
-2828.237691    1
-2636.090517    1
               ..
-7120.517564    1
-2146.884040    1
-881.454684     1
-794.666350     1
-3382.113891    1
Name: days_employed, Length: 19351, dtype: int64

In [22]:
len(df.loc[df['days_employed'] < 0]) / len(df)
# Temukan data yang bermasalah di `days_employed`, jika memang terdapat masalah, dan hitung persentasenya
# Persentase masalahnya sekitar 70% dan nilai yang ada dalam minus.


0.7389547038327526

Hal yang menurut saya sangat mungkin terjadi adalah kesalahan dalam penginputan. Mengingat jumlah data yang banyak, hal ini sangat mungkin dapat terjadi. Nilai pada kolom 'days_employed' tidak seharusnya ada minus karena nilainya bukanlah perhitungan minus seperti kerugian.

In [23]:
df['days_employed'] = df['days_employed'].abs()# Atasi nilai yang bermasalah, jika ada



In [24]:
# Periksa hasilnya - pastikan bahwa masalahnya telah diperbaiki
df['days_employed'].value_counts()

142.276217       1
1849.622944      1
886.253127       1
2539.534295      1
390574.985524    1
                ..
1394.302246      1
2325.720832      1
4086.407828      1
1259.497032      1
1636.419775      1
Name: days_employed, Length: 19351, dtype: int64

In [25]:
df['dob_years'].value_counts(normalize=True)# Periksa `dob_years` untuk nilai yang mencurigakan dan hitung persentasenya



35    0.028664
40    0.028293
41    0.028200
34    0.028014
38    0.027782
42    0.027735
33    0.026992
39    0.026620
31    0.026016
36    0.025784
44    0.025412
29    0.025319
30    0.025087
48    0.024994
37    0.024948
50    0.023879
43    0.023833
32    0.023693
49    0.023600
28    0.023368
45    0.023089
27    0.022904
56    0.022625
52    0.022485
47    0.022300
54    0.022253
46    0.022067
58    0.021417
57    0.021370
53    0.021324
51    0.020813
59    0.020627
55    0.020581
26    0.018955
60    0.017515
25    0.016585
61    0.016492
62    0.016353
63    0.012497
64    0.012311
24    0.012265
23    0.011800
65    0.009013
66    0.008502
22    0.008502
67    0.007758
21    0.005157
0     0.004692
68    0.004599
69    0.003949
70    0.003020
71    0.002695
20    0.002369
72    0.001533
19    0.000650
73    0.000372
74    0.000279
75    0.000046
Name: dob_years, dtype: float64

Ada usia 0 dan 70 tahun ke atas. Tidak mungkin ada orang berusia 0 tahun yang mengajukan kredit, sedangkan dengan orang berusia 70 tahun keatas, umumnya lebih sulit mendapatkan kredit karena faktor umur yang terlalu tua. Tetapi masih ada kemungkinan pengajuannya diterima karena pertimbangan memiliki passive income dan lainnya.

In [26]:
# Atasi masalah pada kolom `dob_years`, jika terdapat masalah
df['dob_years'] = df[df['dob_years'] != 0]['dob_years']

In [27]:
# Periksa hasilnya - pastikan bahwa masalahnya telah diperbaiki
df['dob_years'].value_counts()

35.0    617
40.0    609
41.0    607
34.0    603
38.0    598
42.0    597
33.0    581
39.0    573
31.0    560
36.0    555
44.0    547
29.0    545
30.0    540
48.0    538
37.0    537
50.0    514
43.0    513
32.0    510
49.0    508
28.0    503
45.0    497
27.0    493
56.0    487
52.0    484
47.0    480
54.0    479
46.0    475
58.0    461
57.0    460
53.0    459
51.0    448
59.0    444
55.0    443
26.0    408
60.0    377
25.0    357
61.0    355
62.0    352
63.0    269
64.0    265
24.0    264
23.0    254
65.0    194
66.0    183
22.0    183
67.0    167
21.0    111
68.0     99
69.0     85
70.0     65
71.0     58
20.0     51
72.0     33
19.0     14
73.0      8
74.0      6
75.0      1
Name: dob_years, dtype: int64

In [28]:
df['family_status'].value_counts()# Mari kita lihat nilai untuk kolom ini



married              12380
civil partnership     4177
unmarried             2813
divorced              1195
widow / widower        960
Name: family_status, dtype: int64

In [31]:
df['gender'].value_counts()# Mari kita liat nilai dalam kolom ini

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [32]:
df['gender'] = df[df['gender'] != 'XNA']['gender'] #Atasi nilai-nilai yang bermasalah, jika ada

In [33]:
# Periksa hasilnya - pastikan bahwa masalahnya telah diperbaiki
df['gender'].value_counts()


F    14236
M     7288
Name: gender, dtype: int64

Seperti yang bisa dilihat pada hasil pemeriksaan di atas, kolom gender memiliki nilai 'XNA', yang berarti tidak diketahui apakah pengisi data bergender Female atau Male.
Saya memilih untuk menghapusnya karena 'XNA' tidak bisa dipindahkan ke 'F' ataupun 'M'.

In [34]:
df['income_type'].value_counts()# Mari kita lihat nilai dalam kolom ini

employee                       11119
business                        5085
retiree                         3856
civil servant                   1459
entrepreneur                       2
unemployed                         2
paternity / maternity leave        1
student                            1
Name: income_type, dtype: int64

In [35]:
df['income_type'] = df[df['income_type'] != 'unemployed']['income_type']# Atasi nilai yang bermasalah, jika ada

# Unemployed dihapus karena orang yang sedang menganggur berarti tidak memiliki penghasilan,
# maka dari itu dianggap tidak bisa mengajukan kredit.

In [36]:
# Periksa hasilnya - pastikan bahwa masalahnya telah diperbaiki
df['income_type'].value_counts()


employee                       11119
business                        5085
retiree                         3856
civil servant                   1459
entrepreneur                       2
paternity / maternity leave        1
student                            1
Name: income_type, dtype: int64

Diantara semua 'income type' ada nilai 'unemployed'. Menurut saya, nilai ini perlu dihapus karena seperti penjelasan yang tertulis di atas, orang yang sedang tidak bekerja berarti tidak memiliki penghasilan, tidak berstatus pelajar dan dianggap tidak dapat melakukan pembayaran kredit nantinya.
Berbeda dengan 'student', dari hasil research yang saya lakukan, orang yang berstatus pelajar dapat mengajukan kredit selama ia memiliki atau dapat menunjukkan bukti memiliki penghasilan dari hasil kerja part-time ataupun memiliki wali yang berumur diatas 21 tahun (yang memiliki penghasilan stabil dan akan bertanggung jawab dalam kegagalan pembayaran kredit). 

In [37]:
df['purpose'].value_counts()

wedding ceremony                            797
having a wedding                            777
to have a wedding                           774
real estate transactions                    676
buy commercial real estate                  664
buying property for renting out             653
housing transactions                        653
transactions with commercial real estate    651
purchase of the house                       647
housing                                     647
purchase of the house for my family         641
construction of own property                635
property                                    634
transactions with my real estate            630
building a real estate                      626
buy real estate                             624
building a property                         620
purchase of my own house                    620
housing renovation                          612
buy residential real estate                 607
buying my own car                       

In [16]:
# merapikan value yang mirip atau tujuan yang sama untuk pengajuan kredit
def categorize_purpose(row):
    if 'car' in row:
      return 'car'
    elif 'hous' in row or 'prop' in row or 'real est' in row:
      return 'property'
    elif 'educ' in row or 'uni' in row:
      return 'education'
    else:
      return 'wedding'


df['purpose'] = df['purpose'].apply(categorize_purpose)

In [17]:
# mengecek kembali value dalam kolom purpose
df['purpose'].value_counts()

property     10840
car           4315
education     4022
wedding       2348
Name: purpose, dtype: int64

Anomali yang dapat dilihat pada kolom 'purpose' adalah kemiripan penulisan yang sebenarnya memiliki tujuan yang sama. 
Saya merapikan nilai-nilai tersebut dengan membuat fungsi yang mengategorikan nilai yang sama dalam satu kelompok.

In [19]:
df.duplicated().sum()# Periksa duplikat



341

Jumlah duplikat yang ada pada data yaitu 341, dan akan diperbaiki pada kolom dibawah.

In [23]:
df = df.drop_duplicates()# Atasi duplikat, jika ada

In [24]:
# Lakukan pemeriksaan terakhir untuk mengecek apakah kita memiliki duplikat
df.duplicated().sum()

0

In [26]:
# Periksa ukuran dataset yang sekarang Anda miliki setelah manipulasi pertama yang Anda lakukan
df.shape

(21184, 12)

Pertama, jumlah dataset telah berubah dari 21525 baris menjadi 21184 baris. Perubahan ini dihasilkan dari penghapusan duplikat yang ada sebanyak 341.

# Bekerja dengan nilai yang hilang

In [44]:
# Temukan dictionary
df.to_dict('index')

{0: {'children': 1,
  'days_employed': 8437.673027760233,
  'dob_years': 42.0,
  'education': "bachelor's degree",
  'education_id': 0,
  'family_status': 'married',
  'family_status_id': 0,
  'gender': 'F',
  'income_type': 'employee',
  'debt': 0,
  'total_income': 40620.102,
  'purpose': 'property'},
 1: {'children': 1,
  'days_employed': 4024.803753850451,
  'dob_years': 36.0,
  'education': 'secondary education',
  'education_id': 1,
  'family_status': 'married',
  'family_status_id': 0,
  'gender': 'F',
  'income_type': 'employee',
  'debt': 0,
  'total_income': 17932.802,
  'purpose': 'car'},
 2: {'children': 0,
  'days_employed': 5623.422610230956,
  'dob_years': 33.0,
  'education': 'secondary education',
  'education_id': 1,
  'family_status': 'married',
  'family_status_id': 0,
  'gender': 'M',
  'income_type': 'employee',
  'debt': 0,
  'total_income': 23341.752,
  'purpose': 'property'},
 3: {'children': 3,
  'days_employed': 4124.747206540018,
  'dob_years': 32.0,
  'educ

In [60]:
df['days_employed'].describe()

count     19351.000000
mean      63046.497661
std      140827.311974
min      -18388.949901
25%       -2747.423625
50%       -1203.369529
75%        -291.095954
max      401755.400475
Name: days_employed, dtype: float64

In [61]:
df['total_income'].describe()

count     19351.000000
mean      26787.568355
std       16475.450632
min        3306.762000
25%       16488.504500
50%       23202.870000
75%       32549.611000
max      362496.645000
Name: total_income, dtype: float64

### Memperbaiki nilai yang hilang di `total_income`

Saya akan memperbaiki nilai yang hilang di kolom total income terlebih dulu. Pertama, saya membuat fungsi yang mengategorikan usia, dengan mengategorikannya, akan lebih rapi dan mudah untuk dilihat. Juga karena nilai di kolom dob_years sama pentingnya demgan kolom lain.

Saya akan memulai dengan mengatasi total nilai pendapatan yang hilang. Pertama, membuat kategori usia untuk nasabah. Buat kolom baru yang memuat kategori usia. Strategi ini dapat membantu untuk menghitung total nilai pendapatan.


In [29]:
# Mari kita tulis sebuah fungsi untuk menghitung kategori usia
def age_group(age):
  if age <= 30:
    return 'young adult'
  elif age <= 40:
    return 'adult'
  elif age <= 50:
    return 'middle aged'
  elif age <= 60:
    return 'old'
  else:
    return 'elder'
    

    

Disini saya membuat fungsi untuk mengategorikan usia. Jika orang tersebut berusia kurang dari sama dengan 30 tahun,
maka ia termasuk dalam kategori 'young adult'.
Jika ia berusia kurang dari sama dengan 40 tahun, maka ia termasuk dalam kategori 'adult' dan seterusnya.

In [30]:
# Lakukan pengujian untuk melihat apakah fungsi Anda bekerja atau tidak
print(age_group(35))
print(age_group(47))
print(age_group(65))

adult
middle aged
elder


In [47]:
# Buatlah kolom baru berdasarkan fungsi
df['age_group'] = df['dob_years'].apply(age_group)


In [48]:
# Periksa bagaimana nilai di dalam kolom baru
df.head(5)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,8437.673028,42.0,bachelor's degree,0,married,0,F,employee,0,40620.102,property,middle aged
1,1,4024.803754,36.0,secondary education,1,married,0,F,employee,0,17932.802,car,adult
2,0,5623.422610,33.0,secondary education,1,married,0,M,employee,0,23341.752,property,adult
3,3,4124.747207,32.0,secondary education,1,married,0,M,employee,0,42820.568,education,adult
4,0,340266.072047,53.0,secondary education,1,civil partnership,1,F,retiree,0,25378.572,wedding,old


Selanjutnya saya membuat tabel yang hanya memuat data tanpa nilai yang hilang. Data ini akan digunakan untuk memperbaiki nilai yang hilang.

In [49]:
# Buat tabel tanpa nilai yang hilang dan tampilkan beberapa barisnya untuk memastikan semuanya berjalan dengan baik
df_not_null = df[df['total_income'].notnull()].reset_index(drop=True)
df_not_null.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,8437.673028,42.0,bachelor's degree,0,married,0,F,employee,0,40620.102,property,middle aged
1,1,4024.803754,36.0,secondary education,1,married,0,F,employee,0,17932.802,car,adult
2,0,5623.422610,33.0,secondary education,1,married,0,M,employee,0,23341.752,property,adult
3,3,4124.747207,32.0,secondary education,1,married,0,M,employee,0,42820.568,education,adult
4,0,340266.072047,53.0,secondary education,1,civil partnership,1,F,retiree,0,25378.572,wedding,old
5,0,926.185831,27.0,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,property,young adult
6,0,2879.202052,43.0,bachelor's degree,0,married,0,F,business,0,38484.156,property,middle aged
7,0,152.779569,50.0,secondary education,1,married,0,M,employee,0,21731.829,education,middle aged
8,2,6929.865299,35.0,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,wedding,adult
9,0,2188.756445,41.0,secondary education,1,married,0,M,employee,0,23108.150,property,middle aged


In [50]:
# Perhatikan nilai rata-rata untuk pendapatan berdasarkan faktor yang telah Anda identifikasi
df.groupby(['income_type'])['total_income'].mean()

income_type
business                       32386.793835
civil servant                  27343.729582
employee                       25820.841683
entrepreneur                   79866.103000
paternity / maternity leave     8612.661000
retiree                        21940.394503
student                        15712.260000
Name: total_income, dtype: float64

In [51]:
# Perhatikan nilai median untuk pendapatan berdasarkan faktor yang telah Anda identifikasi
df.groupby(['income_type'])['total_income'].median()

income_type
business                       27577.2720
civil servant                  24071.6695
employee                       22815.1035
entrepreneur                   79866.1030
paternity / maternity leave     8612.6610
retiree                        18962.3180
student                        15712.2600
Name: total_income, dtype: float64

Setelah mencoba beberapa perbandingan, saya memutuskan bahwa perbandingan antara 'income_type' dan 'total_income' adalah yang paling ideal. Saat mengajukan kredit, biasanya anda akan ditanya mengenai pekerjaan dan berapa penghasilan anda. Kedua hal ini memegang peran penting untuk keputusan pihak bank sebagai pemberi kredit.

Saya menggunakan median karena saat mengecek dengan menggunakan boxplot, distribusi nilai pada sebelah kiri lebih menonjol atau miring ke sebelah kiri. Untuk itu, menggunakan median akan lebih baik untuk mengatasi nilai pada 'total_income'.

In [109]:
#  Tulis fungsi yang akan kita gunakan untuk mengisi nilai yang hilang
def fill_missing_value (dataframe, agg_column, value_column):
    grouped_values = dataframe.groupby(agg_column)[value_column].median().reset_index()
    size = len(grouped_values)
    for i in range(size):
        group = grouped_values[agg_column][i]
        value = grouped_values[value_column][i]
        dataframe.loc[(dataframe[agg_column]==group) & (dataframe[value_column].isna()), value_column] = value
    return dataframe

new_df = fill_missing_value(df, 'income_type', 'total_income')

Untuk mengisi nilai yang hilang, saya membuat fungsi seperti di atas. Saya menggunakan median karena pada saat menggunakan boxplot,
bagian data cenderung miring ke kiri. Kolom yang digunakan pada 'group' adalah 'income_type' yang akan diagregasi dan pada 'value' menggunakan kolom 'total_income'.
Kemudian reset_index dilakukan agar menjadi kolom.

for i in range digunakan untuk proses looping jika ada missing value pada kolom 'income_type' maka akan diisi dengan value pada 'total_income'.
Dan pada baris dataframe.loc dilakukan looping dengan kolom yg di agregasi dan jika ada nilai pada kolom pada value yg kosong,
maka akan diisi dari hasil value [(dataframe[agg_column]==group) & (dataframe[value_column].isna()), value_column].
Kemudian reset_index dilakukan agar menjadi kolom.

In [110]:
# Memeriksa bagaimana nilai di dalam kolom baru
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [111]:
# Terapkan fungsi tersebut ke setiap baris
new_df = fill_missing_value(df, 'income_type', 'total_income')

In [112]:
# Periksa apakah kita mendapatkan kesalahan
print(new_df['total_income'].isna().sum())

0


In [113]:
# Periksa jumlah entri di kolom
new_df['total_income'].count()

21525

###  Memperbaiki nilai di `days_employed`

Saya menggunakan median karena saat mengecek dengan menggunakan boxplot, distribusi nilai pada sebelah kiri lebih menonjol atau miring ke sebelah kiri. Untuk itu, menggunakan median akan lebih baik untuk mengatasi nilai pada 'days_employed'.

In [58]:
# Distribusi median dari `days_employed` berdasarkan parameter yang Anda identifikasi
df.groupby(['education'])['days_employed'].median()

education
bachelor's degree      1895.747795
graduate degree        5660.057032
primary education      3043.933615
secondary education    2392.483500
some college           1209.128083
Name: days_employed, dtype: float64

In [59]:
# Distribusi rata-rata dari `days_employed` berdasarkan parameter yang Anda identifikasi
df.groupby(['education'])['days_employed'].mean()

education
bachelor's degree       42375.409174
graduate degree        121323.630206
primary education      130340.426349
secondary education     76413.822372
some college            20656.632017
Name: days_employed, dtype: float64

In [35]:
# Mari tulis fungsi yang menghitung rata-rata atau median (tergantung keputusan Anda) berdasarkan parameter yang Anda identifikasi
def fill_missing_value (dataframe, agg_column, value_column):
    grouped_values = dataframe.groupby(agg_column)[value_column].median().reset_index()
    size = len(grouped_values)
    for i in range(size):
        group = grouped_values[agg_column][i]
        value = round(grouped_values[value_column][i], 0)
        dataframe.loc[(dataframe[agg_column]==group) & (dataframe[value_column].isna()), value_column] = value
    return dataframe

new_df = fill_missing_value(df, 'education', 'days_employed')

Untuk mengisi nilai yang hilang, saya membuat fungsi seperti di atas. Sama seperti mengisi nilai hilang pada 'days_employed',
saya menggunakan median karena pada saat menggunakan boxplot,
bagian data cenderung miring ke kiri. Kolom yang digunakan pada 'group' adalah 'education' yang akan diagregasi dan pada 'value' menggunakan kolom 'days_employed'.
Kemudian reset_index dilakukan agar menjadi kolom.

for i in range digunakan untuk proses looping jika ada missing value pada kolom 'education' maka akan diisi dengan value pada 'days_employed'.
Dan pada baris dataframe.loc dilakukan looping dengan kolom yg di agregasi dan jika ada nilai pada kolom pada value yg kosong,
maka akan diisi dari hasil value [(dataframe[agg_column]==group) & (dataframe[value_column].isna()), value_column].

In [79]:
# Periksa apakah fungsi Anda dapat bekerja
new_df_test


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,high
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,high
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house,high
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,high
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,high
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions,high
21521,0,343937.404131,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car,high
21522,1,-2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property,high
21523,3,-3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car,high


In [105]:
# Terapkan fungsi ke days_employed
new_df = fill_missing_value(df, 'education', 'days_employed')

In [106]:
# Periksa apakah fungsi Anda bekerja
print(new_df['days_employed'].isna().sum())

0


In [107]:
new_df['days_employed'].count()# Periksa entri di semua kolom - pastikan kita memperbaiki semua nilai yang hilang

21525

In [114]:
df.isna().sum() 
# Chamdani

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

## Pengkategorian Data

Untuk menjawab pertanyaan dan menguji hipotesis, saya akan bekerja dengan data yang telah dikategorikan.

In [31]:
# Tampilkan nilai data yang Anda pilih untuk pengkategorian
df['purpose'].value_counts()


property     10616
car           4277
education     3973
wedding       2318
Name: purpose, dtype: int64

[Mari kita memeriksa nilai unik]

In [32]:
# Periksa nilai unik
df['purpose'].unique()

array(['property', 'car', 'education', 'wedding'], dtype=object)

Inilah kelompok utama apakah yang dapat diidentifikasi berdasarkan nilai uniknya.
Ada 4 tujuan pengajuan kredit yaitu property, car, education dan wedding. 
Kita akan mengkategorikan data kita berdasarkan topik ini.


In [42]:
# Mari kita tulis sebuah fungsi untuk mengategorikan data berdasarkan topik umum
def purpose_categorize(purpose):
  if 'hous' in purpose or 'prop' in purpose or 'real est' in purpose:
    return 'investment'
  elif 'car' in  purpose:
    return 'asset'
  else:
    return 'personal'

In [43]:
# Buat kolom yang memuat kategori dan hitung nilainya
df['purpose_category'] = df['purpose'].apply(purpose_categorize)
df['purpose_category'].value_counts()

investment    10616
personal       6291
asset          4277
Name: purpose_category, dtype: int64

In [71]:
# Lihat semua data numerik di kolom yang Anda pilih untuk pengkategorian
df['total_income'].value_counts()

22815.1035    1105
27577.2720     509
18962.3180     414
24071.6695     147
17312.7170       2
              ... 
45484.1090       1
27715.4580       1
23834.5340       1
26124.6130       1
41428.9160       1
Name: total_income, Length: 19350, dtype: int64

# Dapatkan kesimpulan statistik untuk kolomnya
Total income yang memiliki rentang yang beragam dapat disimpulkan bahwa lebih banyak orang yang memiliki penghasilan tinggi yang mengajukan kredit


Rentang kategori ini dibagi menjadi 3 yaitu low, medium, high. Membagi pendapatan menjadi 3 rentang akan memudahkan untuk melihat mana tingkat pendapatan rendah, sedang dan tinggi. 3 rentang dirasa cukup dan sederhana.

In [117]:
# Buat fungsi yang melakukan pengkategorian menjadi kelompok numerik yang berbeda berdasarkan rentang
def total_income_category(total_income):
    if total_income < 10000:
      return 'low'
    elif total_income >= 10000 and total_income <= 1000000:
      return 'medium'
    else:
      return 'high'

In [118]:
# Buat kolom yang memuat kategori
df['total_income_category'] = new_df['total_income'].apply(total_income_category)

In [119]:
# Hitung setiap nilai kategori untuk melihat pendistribusiannya
df['total_income_category'].value_counts()

high      20500
low         926
medium       99
Name: total_income_category, dtype: int64

In [120]:
df['total_income_category'].unique()

array(['high', 'low', 'medium'], dtype=object)

## Memeriksa hipotesis


**Apakah terdapat korelasi antara memiliki anak dengan melakukan pelunasan tepat waktu?**

In [69]:
# Periksa data anak dan data pelunasan tepat waktu
total_credit_children = df.groupby('children')['debt'].count()
debt_credit_children = df.groupby('children')['debt'].sum()

In [71]:
# Hitung gagal bayar berdasarkan jumlah anak
ratio_children = debt_credit_children/total_credit_children
ratio_children.sort_values(ascending=False)

children
 20    0.105263
 4     0.097561
 2     0.094404
 1     0.092154
 3     0.081818
 0     0.075129
-1     0.021277
 5     0.000000
Name: debt, dtype: float64

**Kesimpulan**

Berdasarkan cek data dan perhitungan yang dilakukan, calon peminjam yang memiliki 4 anak memiliki tingkat kegagalan membayar paling tinggi yaitu sekitar 9,7%. Sedangkan untuk calon peminjam yang belum/tidak memiliki anak, memiliki tingkat kegagalan membayar sebanyak sekitar 7,4%. Di luar dugaan, calon peminjam yang memiliki 5 anak, memiliki tingkat kegagalan membayar sebanyak 0%.


**Apakah terdapat korelasi antara status keluarga dengan pelunasan tepat waktu?**

In [72]:
# Periksa data status keluarga dan pelunasan tepat waktu
total_credit_family_status = df.groupby('family_status')['debt'].count()
debt_credit_family_status = df.groupby('family_status')['debt'].sum()

In [73]:
# Hitung gagal bayar berdasarkan status keluarga
ratio_family = debt_credit_family_status/total_credit_family_status
ratio_family.sort_values(ascending=False)

family_status
unmarried            0.097405
civil partnership    0.092890
married              0.075202
divorced             0.071130
widow / widower      0.065625
Name: debt, dtype: float64

**Kesimpulan**

Berdasarkan cek data dan perhitungan yang dilakukan, hasil masing-masing tiap status keluarga memiliki tingkat kegagalan pembayaran yang tinggi. Namun menurut saya sebenarnya status keluarga tidak memiliki korelasi dengan tepat waktu atau tidaknya pelunasan.

**Apakah terdapat korelasi antara tingkat pendapatan dengan membayar kembali tepat waktu?**

In [122]:
# Periksa data tingkat pendapatan dan pelunasan tepat waktu
total_credit_income = df.groupby('total_income_category')['debt'].count()
debt_credit_income = df.groupby('total_income_category')['debt'].sum()


# Hitung gagal bayar berdasarkan tingkat pendapatan
ratio_income = debt_credit_income/total_credit_income
ratio_income.sort_values(ascending=False)

total_income_category
high      0.081805
low       0.062635
medium    0.060606
Name: debt, dtype: float64

**Kesimpulan**

Hasil dari perhitungan menunjukkan calon peminjam yang memiliki pendapatan rendah, sedang dan tinggi sama-sama memiliki tingkat kegagalan membayar yang tidak selisih jauh. 

Menurut saya, tentu ada korelasi antara tingkat pendapatan dengan pelunasan tepat waktu, karena tentunya penghasilan lah yang akan digunakan untuk membayar kredit.

**Bagaimana tujuan kredit memengaruhi tingkat gagal bayar?**

In [80]:
# Periksa persentase tingkat gagal bayar untuk setiap tujuan kredit dan lakukan penganalisisan
total_purpose_credit = df.groupby('purpose')['debt'].count()
debt_credit_purpose = df.groupby('purpose')['debt'].sum()
ratio_purpose = total_purpose_credit/debt_credit_purpose
ratio_purpose.sort_values(ascending=False)

purpose
property     13.861893
wedding      12.623656
education    10.870270
car          10.707196
Name: debt, dtype: float64

**Kesimpulan**

Berdasarkan tujuan kredit, calon peminjam yang berhubungan dengan bidang property memiliki tingkat kegagalan membayar paling tinggi dibandingkan dengan tujuan kredit lainnya, yaitu sekitar 13%. Tentu tidak mengherankan karena harga property yang memang terbilang mahal. Walaupun persentase tingkat kegagalan membayar pada tujuan kredit lainnya memiliki selisih tidak begitu jauh dengan property.


# Kesimpulan umum 


Data ini memiliki nilai yang hilang, nilai yang mempunyai banyak kemiripan dan juga kesalahan penulisan. Hal pertama yang harus dilakukan tentunya adalah pengecekan data agar dapat mengetahui langkah apa yang harus dilakukan setelahnya, seperti memperbaiki kesalahan penulisan, mengelompokkan data yang memiliki kemiripan seperti pada kolom tujuan kredit.


Apakah terdapat korelasi antara memiliki anak dengan melakukan pelunasan tepat waktu?
Bisa jadi ada bisa jadi tidak, seperti yang bisa dilihat pada data perhitungan, orang yang tidak/belum memiliki anak memiliki tingkat gagal membayar sebanyak 7,4% dan angka ini tidak selisih jauh dengan tingkat gagal peminjam yang telah memiliki anak yaitu sekitar 8-9%.


Apakah terdapat korelasi antara status keluarga dengan pelunasan tepat waktu?
Untuk hal ini saya merasa tidak ada korelasinya. Status keluarga seseorang tidak terlalu berpengaruh terhadap tepat waktunya atau tidak untuk membayar pelunasan. Hal ini masih berkaitan dengan tingkat kegagalan peminjam yang belum/tidak memiliki anak dan yang sudah memiliki anak.


Apakah terdapat korelasi antara tingkat pendapatan dengan membayar kembali tepat waktu?
Seperti yang sudah ditulis di atas, menurut saya ini tingkat pendapatan tentunya sangat memiliki korelasi dengan tingkat kegagalan dalam pembayaran. Hal ini dikarenakan peminjam tentunya akan membayar pinjaman dengan penghasilannya. Jika peminjam memiliki penghasilan yang rendah, tentunya akan mengalami kesulitan yang lebih tinggi dalam membayar kredit dibandingkan dengan peminjam yang memiliki penghasilan tinggi.

